In [1]:
import numpy

from cicada.additive import AdditiveProtocolSuite
from cicada.communicator import SocketCommunicator
from cicada.encoding import FixedPoint

def main(communicator):
    encoding = FixedPoint(precision=2)
    protocol = AdditiveProtocolSuite(communicator, order=127, encoding=encoding)
    a_share = protocol.share(src=0, secret=numpy.array(2), shape=())
    b_share = protocol.share(src=1, secret=numpy.array(3), shape=())
    c_share = protocol.add(a_share, b_share)
    c = protocol.reveal(c_share)

Note that we're using a very low-precision encoding (`precision=2`) and a very small field (`order=127`) to make the values in the logged output easier to read.

In [ ]:
import logging

from cicada import transcript

with transcript.record():
    handler = transcript.net_handler()
    transcript.set_handler(logging.getLogger(), handler)
    SocketCommunicator.run(fn=main, world_size=3);

In [ ]:
def main(communicator):
    transcript.log("Let's setup additive sharing!")
    encoding = FixedPoint(precision=2)
    protocol = AdditiveProtocolSuite(communicator, order=127, encoding=encoding)
    transcript.log("Let's share some secrets!")
    a_share = protocol.share(src=0, secret=numpy.array(2), shape=())
    b_share = protocol.share(src=1, secret=numpy.array(3), shape=())
    transcript.log("Let's add some secrets!")
    c_share = protocol.add(a_share, b_share)
    transcript.log("Let's reveal the results!")
    c = protocol.reveal(c_share)

with transcript.record():
    handler = transcript.net_handler()
    transcript.set_handler(logging.getLogger(), handler)
    SocketCommunicator.run(fn=main, world_size=3);

In [ ]:
with transcript.record():
    fmt = "# {processName}: {msg}"
    netfmt = "{processName},{net.verb},{net.src},{net.dst},{net.tag},{net.payload}"
    handler = transcript.net_handler(fmt=fmt, netfmt=netfmt)
    transcript.set_handler(logging.getLogger(), handler)
    SocketCommunicator.run(fn=main, world_size=3);

If you look carefully, you can see that this produces two nearly identical events for each message (once when the message is sent, and once when the message is received).  If you wish to eliminate the duplication, e.g. by only logging messages when they're sent, you can specify that too:

In [ ]:
with transcript.record():
    fmt = "# {processName}: {msg}"
    netfmt = "{processName},{net.verb},{net.src},{net.dst},{net.tag},{net.payload}"
    handler = transcript.net_handler(received=False, fmt=fmt, netfmt=netfmt)
    transcript.set_handler(logging.getLogger(), handler)
    SocketCommunicator.run(fn=main, world_size=3);

In [2]:
from cicada.arithmetic import field

with transcript.record():
    handler = transcript.code_handler()
    transcript.set_handler(logging.getLogger(), handler)
    
    f = field(order=127)
    a = f.ones(3)
    b = f.uniform(size=3, generator=numpy.random.default_rng())
    a += b

NameError: name 'transcript' is not defined

In [ ]:
import io

with transcript.record():
    buffer = io.StringIO()
    handler = transcript.code_handler(logging.StreamHandler(buffer))
    transcript.set_handler(logging.getLogger(), handler)
    
    f = Field(order=127)
    a = f.ones(3)
    b = f.uniform(size=3, generator=numpy.random.default_rng())
    f.inplace_add(a, b)

In [ ]:
buffer.seek(0)
for line in buffer:
    print(line.strip())

Manual inspection shows that the transcript contains the expected output. Now, we can execute the buffered transcript, one-line-at-a-time, and confirm that none of the assertions raise exceptions:

In [ ]:
import cicada.transcript

try:
    buffer.seek(0)
    for line in buffer:
        exec(line)
except Exception as e:
    print(f"Consistency verification failed: {e}")
else:
    print("Consistency verification succeeded.")

Let's go back to our original example. Because it requires three players, we'll setup Cicada to write a separate transcript for each player:  

In [ ]:
def main(comm):
    handler = transcript.code_handler(logging.FileHandler(f"player-{comm.rank}.py", "w"))
    transcript.set_handler(logging.getLogger(), handler)

    transcript.log("Let's setup additive sharing!")
    encoding = FixedPoint(precision=2)
    protocol = AdditiveProtocolSuite(comm, order=127, encoding=encoding)
    transcript.log("Let's share some secrets!")
    a_share = protocol.share(src=0, secret=numpy.array(2), shape=())
    b_share = protocol.share(src=1, secret=numpy.array(3), shape=())
    transcript.log("Let's add some secrets!")
    c_share = protocol.add(a_share, b_share)
    transcript.log("Let's reveal the results!")
    c = protocol.reveal(c_share)

with transcript.record():
    SocketCommunicator.run(fn=main, world_size=3);

Note that we've moved our transcript configuration code inside `main`, which allows us to generate a separate filename for each transcript based on player rank. Also note that we've left our context messages in-place. Let's see what the results look like for one of the players:

In [ ]:
with open("player-0.py") as stream:
    for line in stream:
        print(line.strip())

In [ ]:
import traceback

for rank in range(3):
    try:
        with open(f"player-{rank}.py") as stream:
            for line in stream:
                exec(line)
    except Exception as e:
        print(f"Player {rank} consistency verification failed: {e}")
        print(traceback.format_exc())
    else:
        print(f"Player {rank} consistency verification succeeded.")

In [ ]:
def main(comm):
    handler = logging.StreamHandler() if comm.rank == 0 else logging.NullHandler()
    handler = transcript.code_handler(handler, sent=True, received=True)
    transcript.set_handler(logging.getLogger(), handler)

    transcript.log("Let's setup additive sharing!")
    encoding = FixedPoint(precision=2)
    protocol = AdditiveProtocolSuite(comm, order=127, encoding=encoding)
    transcript.log("Let's share some secrets!")
    a_share = protocol.share(src=0, secret=numpy.array(2), shape=())
    b_share = protocol.share(src=1, secret=numpy.array(3), shape=())
    transcript.log("Let's add some secrets!")
    c_share = protocol.add(a_share, b_share)
    transcript.log("Let's reveal the results!")
    c = protocol.reveal(c_share)

with transcript.record():
    SocketCommunicator.run(fn=main, world_size=3);